In [5]:
from datetime import datetime
import lxml
from lxml import html
import requests
import numpy as np
import pandas as pd

In [6]:
def get_page(url):
    return requests.get(url)

In [7]:
def parse_rows(table_rows):
    parsed_rows = []

    for table_row in table_rows:
        parsed_row = []
        el = table_row.xpath("./div")

        none_count = 0

        for rs in el:
            try:
                (text,) = rs.xpath('.//span/text()[1]')
                parsed_row.append(text)
            except ValueError:
                parsed_row.append(np.NaN)
                none_count += 1

        if (none_count < 4):
            parsed_rows.append(parsed_row)
            
    return pd.DataFrame(parsed_rows)

In [8]:
def clean_data(df):
    df = df.set_index(0) 
    df = df.transpose()
    
    cols = list(df.columns)
    cols[0] = 'Date'
    df = df.set_axis(cols, axis='columns', inplace=False)
    
    numeric_columns = list(df.columns)[1::] 

    for column_index in range(1, len(df.columns)): 
        df.iloc[:,column_index] = df.iloc[:,column_index].str.replace(',', '') 
        df.iloc[:,column_index] = df.iloc[:,column_index].astype(np.float64)
        
    return df

In [9]:
def scrape_table(url):
    page = get_page(url);
    tree = html.fromstring(page.content)
    table_rows = tree.xpath("//div[contains(@class, 'D(tbr)')]")    
    df = parse_rows(table_rows)
    df = clean_data(df)
    return df

In [10]:
symbol = 'AAPL'
df_balance_sheet = scrape_table('https://finance.yahoo.com/quote/' + symbol + '/balance-sheet?p=' + symbol)

In [11]:
df_balance_sheet

,Date,Total Assets,Total Liabilities Net Minority Interest,Total Equity Gross Minority Interest,Total Capitalization,Common Stock Equity,Net Tangible Assets,Working Capital,Invested Capital,Tangible Book Value,Total Debt,Net Debt,Share Issued,Ordinary Shares Number
1,9/30/2019,338516000.0,248028000.0,90488000.0,182295000.0,90488000.0,90488000.0,57101000.0,198535000.0,90488000.0,108047000.0,59203000.0,4443236.0,4443236.0
2,9/30/2018,365725000.0,258578000.0,107147000.0,200882000.0,107147000.0,107147000.0,14473000.0,221630000.0,107147000.0,114483000.0,88570000.0,4754986.0,4754986.0
3,9/30/2017,375319000.0,241272000.0,134047000.0,231254000.0,134047000.0,126032000.0,27831000.0,249727000.0,126032000.0,115680000.0,95391000.0,5126201.0,5126201.0
4,9/30/2016,321686000.0,193437000.0,128249000.0,203676000.0,128249000.0,119629000.0,27863000.0,215281000.0,119629000.0,87032000.0,66548000.0,5336166.0,5336166.0


In [12]:
df_income_statement = scrape_table('https://finance.yahoo.com/quote/' + symbol + '/financials?p=' + symbol)


In [14]:
df_income_statement

,Date,Total Revenue,Cost of Revenue,Gross Profit,Operating Expense,Operating Income,Net Non Operating Interest Income Expense,Other Income Expense,Pretax Income,Tax Provision,...,Interest Income,Interest Expense,Net Interest Income,EBIT,Reconciled Cost of Revenue,Reconciled Depreciation,Net Income from Continuing Operation Net Minority Interest,Normalized EBITDA,Tax Rate for Calcs,Tax Effect of Unusual Items
1,ttm,273857000.0,169277000.0,104580000.0,37442000.0,67138000.0,1052000.0,127000.0,68317000.0,9893000.0,...,4101000.0,3049000.0,1052000.0,71366000.0,169277000.0,11533000.0,58424000.0,82899000.0,0.0,0.0
2,9/30/2019,260174000.0,161782000.0,98392000.0,34462000.0,63930000.0,1385000.0,422000.0,65737000.0,10481000.0,...,4961000.0,3576000.0,1385000.0,69313000.0,161782000.0,12547000.0,55256000.0,81860000.0,0.0,0.0
3,9/30/2018,265595000.0,163756000.0,101839000.0,30941000.0,70898000.0,2446000.0,-441000.0,72903000.0,13372000.0,...,5686000.0,3240000.0,2446000.0,76143000.0,163756000.0,10903000.0,59531000.0,87046000.0,0.0,0.0
4,9/30/2017,229234000.0,141048000.0,88186000.0,26842000.0,61344000.0,2878000.0,-133000.0,64089000.0,15738000.0,...,5201000.0,2323000.0,2878000.0,66412000.0,141048000.0,10157000.0,48351000.0,76569000.0,0.0,0.0
5,9/30/2016,215639000.0,131376000.0,84263000.0,24239000.0,60024000.0,2543000.0,-1195000.0,61372000.0,15685000.0,...,3999000.0,1456000.0,2543000.0,62828000.0,131376000.0,10505000.0,45687000.0,73333000.0,0.0,0.0


In [13]:
df_cashflow_statement = scrape_table('https://finance.yahoo.com/quote/' + symbol + '/cash-flow?p=' + symbol)


In [15]:
df_cashflow_statement

,Date,Operating Cash Flow,Investing Cash Flow,Financing Cash Flow,End Cash Position,Income Tax Paid Supplemental Data,Interest Paid Supplemental Data,Capital Expenditure,Issuance of Capital Stock,Issuance of Debt,Repayment of Debt,Repurchase of Capital Stock,Free Cash Flow
1,ttm,80008000.0,-10618000.0,-86502000.0,35039000.0,11878000.0,3135000.0,-8302000.0,820000.0,22794000.0,-13908000.0,-72615000.0,71706000.0
2,9/30/2019,69391000.0,45896000.0,-90976000.0,50224000.0,15263000.0,3423000.0,-10495000.0,781000.0,6963000.0,-8805000.0,-66897000.0,58896000.0
3,9/30/2018,77434000.0,16066000.0,-87876000.0,25913000.0,10417000.0,3022000.0,-13313000.0,669000.0,6969000.0,-6500000.0,-72738000.0,64121000.0
4,9/30/2017,63598000.0,-46446000.0,-17347000.0,20289000.0,11591000.0,2092000.0,-12795000.0,555000.0,28662000.0,-3500000.0,-32900000.0,50803000.0
5,9/30/2016,65824000.0,-45977000.0,-20483000.0,20484000.0,10444000.0,1316000.0,-13548000.0,495000.0,24954000.0,-2500000.0,-29722000.0,52276000.0


### Define one function that scraps everything and puts in a Single Dataframe for a given ticker

In [23]:
def scrape(symbol):
    print('Attempting to scrape data for ' + symbol)

    df_balance_sheet = scrape_table('https://finance.yahoo.com/quote/' + symbol + '/balance-sheet?p=' + symbol)
    df_balance_sheet = df_balance_sheet.set_index('Date')

    df_income_statement = scrape_table('https://finance.yahoo.com/quote/' + symbol + '/financials?p=' + symbol)
    df_income_statement = df_income_statement.set_index('Date')
    
    df_cash_flow = scrape_table('https://finance.yahoo.com/quote/' + symbol + '/cash-flow?p=' + symbol)
    df_cash_flow = df_cash_flow.set_index('Date')
    
    df_joined = df_balance_sheet \
        .join(df_income_statement, on='Date', how='outer', rsuffix=' - Income Statement') \
        .join(df_cash_flow, on='Date', how='outer', rsuffix=' - Cash Flow') \
        .dropna(axis=1, how='all') \
        .reset_index()
            
    df_joined.insert(1, 'Symbol', symbol)
    print('Successfully scraped data for ' + symbol)
    return df_joined
    

In [24]:
financial_data_reliance = scrape('RELIANCE.NS')

Attempting to scrape data for RELIANCE.NS
Successfully scraped data for RELIANCE.NS


In [25]:
financial_data_aapl

,index,Symbol,Date,Total Assets,Total Liabilities Net Minority Interest,Total Equity Gross Minority Interest,Total Capitalization,Common Stock Equity,Net Tangible Assets,Working Capital,...,Financing Cash Flow,End Cash Position,Income Tax Paid Supplemental Data,Interest Paid Supplemental Data,Capital Expenditure,Issuance of Capital Stock,Issuance of Debt,Repayment of Debt,Repurchase of Capital Stock,Free Cash Flow
0,9/30/2019,AAPL,9/30/2019,338516000.0,248028000.0,90488000.0,182295000.0,90488000.0,90488000.0,57101000.0,...,-90976000.0,50224000.0,15263000.0,3423000.0,-10495000.0,781000.0,6963000.0,-8805000.0,-66897000.0,58896000.0
1,9/30/2018,AAPL,9/30/2018,365725000.0,258578000.0,107147000.0,200882000.0,107147000.0,107147000.0,14473000.0,...,-87876000.0,25913000.0,10417000.0,3022000.0,-13313000.0,669000.0,6969000.0,-6500000.0,-72738000.0,64121000.0
2,9/30/2017,AAPL,9/30/2017,375319000.0,241272000.0,134047000.0,231254000.0,134047000.0,126032000.0,27831000.0,...,-17347000.0,20289000.0,11591000.0,2092000.0,-12795000.0,555000.0,28662000.0,-3500000.0,-32900000.0,50803000.0
3,9/30/2016,AAPL,9/30/2016,321686000.0,193437000.0,128249000.0,203676000.0,128249000.0,119629000.0,27863000.0,...,-20483000.0,20484000.0,10444000.0,1316000.0,-13548000.0,495000.0,24954000.0,-2500000.0,-29722000.0,52276000.0
4,NaN,AAPL,ttm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-86502000.0,35039000.0,11878000.0,3135000.0,-8302000.0,820000.0,22794000.0,-13908000.0,-72615000.0,71706000.0


### Scrape for a list of symbols

In [26]:
def scrape_multi_symbols(symbols):
    return pd.concat([scrape(symbol) for symbol in symbols], sort=False)

In [29]:
scrape_multi_symbols(['MSFT','TSLA'])

Attempting to scrape data for MSFT
Successfully scraped data for MSFT
Attempting to scrape data for TSLA
Successfully scraped data for TSLA


,index,Symbol,Date,Total Assets,Total Liabilities Net Minority Interest,Total Equity Gross Minority Interest,Total Capitalization,Common Stock Equity,Capital Lease Obligations,Net Tangible Assets,...,Financing Cash Flow,End Cash Position,Capital Expenditure,Issuance of Capital Stock,Issuance of Debt,Repayment of Debt,Repurchase of Capital Stock,Free Cash Flow,Income Tax Paid Supplemental Data,Interest Paid Supplemental Data
0,6/30/2020,MSFT,6/30/2020,301311000.0,183007000.0,118304000.0,177882000.0,118304000.0,7671000.0,67915000.0,...,-46031000.0,13576000.0,-15441000.0,1343000.0,0.0,-5518000.0,-22968000.0,45234000.0,NaN,NaN
1,6/30/2019,MSFT,6/30/2019,286556000.0,184226000.0,102330000.0,168992000.0,102330000.0,6188000.0,52554000.0,...,-36887000.0,11356000.0,-13925000.0,1142000.0,0.0,-4000000.0,-19543000.0,38260000.0,NaN,NaN
2,6/30/2018,MSFT,6/30/2018,258848000.0,176130000.0,82718000.0,154960000.0,82718000.0,5568000.0,38982000.0,...,-33590000.0,11946000.0,-11632000.0,1002000.0,7183000.0,-10060000.0,-10721000.0,32252000.0,NaN,NaN
3,6/30/2017,MSFT,6/30/2017,241086000.0,168692000.0,72394000.0,148467000.0,72394000.0,NaN,27166000.0,...,8408000.0,7663000.0,-8129000.0,772000.0,44344000.0,-7922000.0,-11788000.0,31378000.0,NaN,NaN
4,NaN,MSFT,ttm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-46031000.0,13576000.0,-15441000.0,1343000.0,0.0,-5518000.0,-22968000.0,45234000.0,NaN,NaN
0,12/31/2019,TSLA,12/31/2019,34309000.0,26842000.0,7467000.0,18252000.0,6618000.0,1050000.0,6081000.0,...,1529000.0,6783000.0,-1437000.0,848000.0,10669000.0,-9871000.0,NaN,968000.0,54000.0,455000.0
1,12/31/2018,TSLA,12/31/2018,29739614.0,23981974.0,5757640.0,14326915.0,4923243.0,1855900.0,4572592.0,...,573755.0,4276388.0,-2319516.0,0.0,6176173.0,-6087029.0,NaN,-221714.0,35409.0,380836.0
2,12/31/2017,TSLA,12/31/2017,28655372.0,23420784.0,5234588.0,13655631.0,4237242.0,1801010.0,3815503.0,...,4414864.0,3367914.0,-4081354.0,400175.0,7649376.0,-4263788.0,NaN,-4142008.0,65695.0,182571.0
3,12/31/2016,TSLA,12/31/2016,22664076.0,17125990.0,5538086.0,10731195.0,4752911.0,1459684.0,4376766.0,...,3743976.0,3393216.0,-1440471.0,1701734.0,3622673.0,-1904483.0,NaN,-1564300.0,16385.0,38693.0
4,NaN,TSLA,ttm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2870000.0,9089000.0,-1906000.0,2309000.0,10607000.0,-10357000.0,NaN,799000.0,NaN,NaN
